In [253]:
# imports
#!pip install tables
#!pip install fastparquet
from fastparquet import write

from tables import *
import openai  # for generating embeddings
import pandas as pd  # for DataFrames to store article sections and embeddings
import re  # for cutting <ref> links out of Wikipedia articles
import tiktoken  # for counting tokens
import numpy as np
from dotenv import dotenv_values

In [254]:
#To use embeddings you must create an .env file where the content is OPENAI_API_KEY = "your-api-key"
config = dotenv_values(".env")["OPENAI_API_KEY"]
openai.organization = "org-3O7bHGD9SwjHVDuUCNCGACC3"
openai.api_key = config

In [255]:
GPT_MODEL = "gpt-3.5-turbo"  # only matters insofar as it selects which tokenizer to use
EMBEDDING_MODEL = "text-embedding-ada-002"

In [256]:
df = pd.read_csv("legal_acts_estonia.csv", names=['type', 'nr','text','link'])

In [257]:
df

,type,nr,text,link
0,VVS,para1,§ 1.\nVabariigi Valitsuse pädevus\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para1
1,VVS,para2,§ 2.\nVabariigi Valitsuse asukoht\nVabariigi V...,https://www.riigiteataja.ee/akt/VVS#para2
2,VVS,para3,§ 3.\nVabariigi Valitsuse liikmed\n(1) Vabarii...,https://www.riigiteataja.ee/akt/VVS#para3
3,VVS,para3b1,§ 3.1.\nVabariigi Valitsuse liikme juurdepääs ...,https://www.riigiteataja.ee/akt/VVS#para3b1
4,VVS,para4,§ 4.\nVabariigi Valitsuse liikmete tööülesanne...,https://www.riigiteataja.ee/akt/VVS#para4
...,...,...,...,...
32784,NaN,para7,§ 7.\nKaitsekulu\nKaitsekulu on NATO meetodi j...,https://www.riigiteataja.ee/akt/#para7
32785,NaN,para8,§ 8.\nEesti Haigekassa eelarvepositsioon\nEest...,https://www.riigiteataja.ee/akt/#para8
32786,NaN,para9,§ 9.\nEelarveaasta jooksul riigiteede üleandmi...,https://www.riigiteataja.ee/akt/#para9
32787,NaN,para10,§ 10.\nTapa spordi- ja vabaajakeskuse ehituse ...,https://www.riigiteataja.ee/akt/#para10


In [258]:
df["text"][0]

'§\xa01.\nVabariigi Valitsuse pädevus\n(1) Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.\n(2) Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.\n'

In [259]:
df["title"] = df["text"].str.split("\n").str[1]
df["text"] = df["text"].str.split("\n").apply(lambda x: ','.join(x[2:]))

In [260]:
df["text"] = df["text"].str.replace('\n','')

In [261]:
#have to split up to have less than 1600 words
df['split_text'] = df['text'].str.split(r'\(\d+\)')


In [262]:
df["split_text"][0]

['',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.,',
 ' Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.,']

In [263]:
df = df.explode("split_text")

In [264]:
df = df[df["split_text"]!= ""]


In [265]:
def count_words(text):
    return len(text.split())
df['word_count'] = df['split_text'].apply(count_words)


In [266]:
df_cut = df[["type","nr","link","title","split_text"]]
over_length = df[df["word_count"]>600]
#remove the long strings for now
df = df[df["word_count"]<600]


In [267]:
over_length['split_text_2'] = over_length['split_text'].str.split(r'\(\d+\.\d+\)')
over_length = over_length.explode("split_text_2")

In [268]:
over_length['word_count'] = over_length['split_text_2'].apply(count_words)

In [269]:
#if they still have too many words, most often the paragraphs are lists of definitions, which can be split by list enumeration
#other with a shorter length can be added back to original dataframe
over_length_merge = over_length[over_length["word_count"]<600]
over_length_merge = over_length_merge.drop(columns=["split_text"]).rename(columns={"split_text_2":"split_text"})


In [270]:
df = df.append(over_length_merge)

C:\Users\annilo\AppData\Local\Temp\ipykernel_17700\605463739.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(over_length_merge)


In [271]:
#
over_length = over_length[over_length["word_count"]>600]
over_length["title_2"] = over_length["split_text_2"].str.split(r',\d+[\.\d+]*\)').str[0]
over_length["split_text_3"] = over_length["split_text_2"].str.split(r',\d+[\.\d+]*\)')
over_length = over_length.explode("split_text_3")


In [272]:
over_length = over_length[over_length["title_2"]!=over_length["split_text_3"]]
over_length['word_count'] = over_length['split_text_3'].apply(count_words)
over_length["title"] = over_length["title"]+ '. ' + over_length["title_2"]
over_length = over_length.drop(columns=["split_text","split_text_2","title_2"]).rename(columns={"split_text_3":"split_text"})


In [273]:
df = df.append(over_length)

C:\Users\annilo\AppData\Local\Temp\ipykernel_17700\977430784.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(over_length)


,type,nr,text,link,title,split_text,word_count,split_text_2,split_text_3


In [276]:
df["concatenated"] = df["type"]+df["nr"]+df["title"]+ " "+ df["split_text"]
df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


C:\Users\annilo\AppData\Local\Temp\ipykernel_17700\1343037904.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["concatenated"] = df["concatenated"].str.replace(r'\s+', ' ').str.rstrip(",")


In [24]:
#replace all list enumeration
#dont think it is needed
#df["split_text"] = df["split_text"].str.replace(r',\d+[\.\d+]*\)','')

,type,nr,text,link,title,split_text,word_count,concatenated
965,VPTS,para47,"(1) Käesoleva seaduse III osa ei kohaldata:,1)...",https://www.riigiteataja.ee/akt/VPTS#para47,Käesoleva osa kohaldamine,"Käesoleva seaduse III osa ei kohaldata:,1) le...",748,Käesoleva osa kohaldamine Käesoleva seaduse II...
1344,VPTS,para235,"Inspektsioonil on õigus ettekirjutusega:,1) ke...",https://www.riigiteataja.ee/akt/VPTS#para235,Õigused ettekirjutuse tegemisel,"Inspektsioonil on õigus ettekirjutusega:,1) ke...",759,Õigused ettekirjutuse tegemisel Inspektsioonil...
1605,VÕS,para42,"(1) Tüüptingimus on tühine, kui see lepingu ol...",https://www.riigiteataja.ee/akt/VÕS#para42,Tüüptingimuse tühisus,"Lepingus, mille teiseks pooleks on tarbija, o...",842,"Tüüptingimuse tühisus Lepingus, mille teiseks ..."
1987,VÕS,para380,(1) Ettevõtja esitab tarbijale mõistliku aja j...,https://www.riigiteataja.ee/akt/VÕS#para380,Lepingueelne teave ja reklaam,Ettevõtja esitab tarbijale mõistliku aja jook...,691,Lepingueelne teave ja reklaam Ettevõtja esitab...
1987,VÕS,para380,(1) Ettevõtja esitab tarbijale mõistliku aja j...,https://www.riigiteataja.ee/akt/VÕS#para380,Lepingueelne teave ja reklaam,Kui kasutusõigus puudutab projekteeritavat võ...,982,Lepingueelne teave ja reklaam Kui kasutusõigus...
4917,TuMS,para13,(1) Tulumaksuga maksustatakse kõik töö eest sa...,https://www.riigiteataja.ee/akt/TuMS#para13,Palgatulu,"Tulumaksuga ei maksustata:,1) ametnikule, töö...",1059,"Palgatulu Tulumaksuga ei maksustata:,1) ametni..."
4979,TuMS,para53,"(1) Mitteresidendist juriidiline isik, kellel ...",https://www.riigiteataja.ee/akt/TuMS#para53,Mitteresidendist juriidilise isiku Eestis asuv...,"Paragrahvi 50, 50.1 või 50.2 alusel maksustat...",655,Mitteresidendist juriidilise isiku Eestis asuv...
7020,TMS,para2,(1) Käesoleva seadustiku alusel täidetakse nõu...,https://www.riigiteataja.ee/akt/TMS#para2,Täitedokumendid,Käesoleva seadustiku alusel täidetakse nõuded...,783,Täitedokumendid Käesoleva seadustiku alusel tä...
7454,TKS,para16,(1) Eksitav kauplemisvõte on nii eksitav tegev...,https://www.riigiteataja.ee/akt/TKS#para16,Eksitav kauplemisvõte,Järgmisi kauplemisvõtteid peetakse alati eksi...,681,Eksitav kauplemisvõte Järgmisi kauplemisvõttei...
8952,SMS,para6,(1) Riik või avalik-õiguslik juriidiline isik ...,https://www.riigiteataja.ee/akt/SMS#para6,Sotsiaalmaksu maksmise erijuhud,Riik või avalik-õiguslik juriidiline isik mak...,954,Sotsiaalmaksu maksmise erijuhud Riik või avali...


In [277]:
laws = np.array(df["concatenated"])
#laws = laws[:10000]

In [278]:
laws

array(['Vabariigi Valitsuse pädevus Vabariigi Valitsus teostab täidesaatvat riigivõimu Eesti Vabariigi põhiseaduse ja seaduste alusel.',
       'Vabariigi Valitsuse pädevus Vabariigi Valitsus teostab täidesaatvat riigivõimu vahetult või valitsusasutuste kaudu.',
       'Vabariigi Valitsuse asukoht Vabariigi Valitsuse asukoht on Tallinnas.',
       ...,
       'Aktsiisilaopidaja ja registreeritud kaubasaaja kohustused. Aktsiisilaopidaja ja registreeritud kaubasaaja, välja arvatud käesoleva paragrahvi lõikes 2 nimetatud juhul, on kohustatud: pidama aktsiisilaost lähetatud denatureeritud alkoholi ja selles sisaldunud etanooli koguselist arvestust alkoholi denatureerimisliigi ja saajate lõikes;,[RT I 2008, 49, 272 - jõust. 01.01.2009]',
       'Aktsiisilaopidaja ja registreeritud kaubasaaja kohustused. Aktsiisilaopidaja ja registreeritud kaubasaaja, välja arvatud käesoleva paragrahvi lõikes 2 nimetatud juhul, on kohustatud: [kehtetu - RT I, 03.04.2018, 2 - jõust. 01.02.2019]',
       'Akts

In [279]:
# calculate embeddings

##
##DO NOT RUN UNLESS NEED TO CREATE NEW EMBEDDINGS (THIS CODE COSTS ABT 2 DOLLARS)

#EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
#BATCH_SIZE = 1000  # you can submit up to 2048 embedding inputs per request
#
#wikipedia_strings = laws.tolist()
#
#embeddings = []
#for batch_start in range(0, len(wikipedia_strings), BATCH_SIZE):
#    batch_end = batch_start + BATCH_SIZE
#    batch = wikipedia_strings[batch_start:batch_end]
#    print(f"Batch {batch_start} to {batch_end-1}")
#    response = openai.Embedding.create(model=EMBEDDING_MODEL, input=batch)
#    for i, be in enumerate(response["data"]):
#        assert i == be["index"]  # double check embeddings are in same order as input
#    batch_embeddings = [e["embedding"] for e in response["data"]]
#    embeddings.extend(batch_embeddings)
#
#result = pd.DataFrame({"text": wikipedia_strings, "embedding": embeddings})

Batch 0 to 999
Batch 1000 to 1999
Batch 2000 to 2999
Batch 3000 to 3999
Batch 4000 to 4999
Batch 5000 to 5999
Batch 6000 to 6999
Batch 7000 to 7999
Batch 8000 to 8999
Batch 9000 to 9999
Batch 10000 to 10999
Batch 11000 to 11999
Batch 12000 to 12999
Batch 13000 to 13999
Batch 14000 to 14999
Batch 15000 to 15999
Batch 16000 to 16999
Batch 17000 to 17999
Batch 18000 to 18999
Batch 19000 to 19999
Batch 20000 to 20999
Batch 21000 to 21999
Batch 22000 to 22999
Batch 23000 to 23999
Batch 24000 to 24999
Batch 25000 to 25999
Batch 26000 to 26999
Batch 27000 to 27999
Batch 28000 to 28999
Batch 29000 to 29999
Batch 30000 to 30999
Batch 31000 to 31999
Batch 32000 to 32999
Batch 33000 to 33999
Batch 34000 to 34999
Batch 35000 to 35999
Batch 36000 to 36999
Batch 37000 to 37999
Batch 38000 to 38999
Batch 39000 to 39999
Batch 40000 to 40999
Batch 41000 to 41999
Batch 42000 to 42999
Batch 43000 to 43999
Batch 44000 to 44999
Batch 45000 to 45999
Batch 46000 to 46999
Batch 47000 to 47999
Batch 48000 to 4

In [ ]:
#df['text'] = df['text'].astype('str')
#df['embedding'] = df['embedding'].astype('str')

In [280]:
#loading is super slow
result.to_hdf(r'embeddedfile_all.h5', key='stage', mode='w')

C:\Users\annilo\AppData\Local\Temp\ipykernel_17700\1531630278.py:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['text', 'embedding'], dtype='object')]

  result.to_hdf(r'embeddedfile_all.h5', key='stage', mode='w')


In [281]:
reread = pd.read_hdf('./embeddedfile_all.h5')  

In [282]:
#reread = reread[:1000]

In [283]:
#reread.to_csv("sample_embeddings.csv")

In [284]:
reread

,text,embedding
0,Vabariigi Valitsuse pädevus Vabariigi Valitsus...,"[0.010672735050320625, -0.034033335745334625, ..."
1,Vabariigi Valitsuse pädevus Vabariigi Valitsus...,"[0.002412177622318268, -0.04879864677786827, -..."
2,Vabariigi Valitsuse asukoht Vabariigi Valitsus...,"[-0.0087286327034235, -0.03157782554626465, -0..."
3,Vabariigi Valitsuse liikmed Vabariigi Valitsus...,"[-0.0025814843829721212, -0.0328136682510376, ..."
4,Vabariigi Valitsuse liikmed Ministrite pädevus...,"[0.003382223192602396, -0.04689347371459007, -..."
...,...,...
107671,Aktsiisilaopidaja ja registreeritud kaubasaaja...,"[-0.008601510897278786, -0.02166212722659111, ..."
107672,Aktsiisilaopidaja ja registreeritud kaubasaaja...,"[-0.007153782062232494, -0.004592832177877426,..."
107673,Aktsiisilaopidaja ja registreeritud kaubasaaja...,"[-0.0041351462714374065, -0.033257536590099335..."
107674,Aktsiisilaopidaja ja registreeritud kaubasaaja...,"[-0.01231425255537033, -0.023286283016204834, ..."


In [285]:
# convert embeddings from CSV str type back to list type
#!pip install ast
import ast  # for converting embeddings saved as strings back to arrays
#reread['embedding'] = reread['embedding'].apply(ast.literal_eval)
df = reread
from scipy import spatial  # for calculating vector similarities for search

In [286]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [287]:


# examples
strings, relatednesses = strings_ranked_by_relatedness("kohtumäärus", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)



relatedness=0.857


'Kohtumäärus Määrusele kohaldatakse vastavalt otsuse kohta sätestatut, kui seadusest või määruse olemusest ei tulene teisiti.,[RT I 2008, 59, 330 - jõust. 01.01.2009]'

relatedness=0.852


'Kohtumäärus Menetlusküsimusi otsustab Riigikohus määrusega.'

relatedness=0.852


'Kohtumäärus Kohus määrab alaealisele eestkostja määrusega.'

relatedness=0.849


'Kohtuistungi määramine Kohtuistungi aja määramisel küsitakse ja arvestatakse võimaluse korral menetlusosaliste arvamust.'

relatedness=0.846


'Vahekohtuniku määramine kohtu poolt Kohus arvestab vahekohtunikku määrates:,1) poolte vahel vahekohtuniku suhtes kokkulepitud tingimusi;,2) asjaolusid, mis tagavad sõltumatu, erapooletu ja kompetentse vahekohtuniku määramise.'

In [291]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = 'Kasuta järgnevaid seaduseid, et vastata küsimustele'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n..\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Sa vastad Eesti seaduste andmebaasi küsimustele."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

In [294]:
ask("Milline on lahutuse vormistamise protsess Eestis?")

'Abielu lahutamine notari juures ja abielu lahutamisele kohaldatava õiguse kokkuleppe sõlmimine. Kohtumenetluses abielu lahutamisele kohaldatava õiguse kohta kokkuleppe sõlmimine märgitakse protokolli. Protokolli märkimine asendab notariaalse tõestamise vormi.'